In [1]:
import numpy as np
from tqdm import tqdm
import cv2

n = 25000
img_width = 299
img_height = 299

X = np.zeros((n,img_width,img_height,3),dtype=np.uint8)
y = np.zeros((n,1),dtype = np.uint8)

for i in tqdm(range(n//2)):
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' %i),(img_width,img_height))
    X[i+n//2] = cv2.resize(cv2.imread('train/dog.%d.jpg' %i),(img_width,img_height))

y[n//2:] = 1

100%|██████████| 12500/12500 [00:51<00:00, 240.39it/s]


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

In [3]:
from keras.models import *
from keras.applications import *
from keras.layers import *
from keras.preprocessing import *

Using TensorFlow backend.


In [4]:
base_model = Xception(input_tensor=Input((img_width,img_height,3)),weights='imagenet',include_top=False,pooling='avg')

In [5]:
NB_LAYER_FREEZE = len(base_model.layers) - 1
for layer in base_model.layers[:NB_LAYER_FREEZE]:
    layer.trainable = False
for layer in base_model.layers[NB_LAYER_FREEZE:]:
    layer.trainable = True
    
x = Dropout(0.25)(base_model.output)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(X_train,y_train, batch_size=16, epochs=5, validation_data=(X_valid,y_valid))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 203s - loss: 0.0675 - acc: 0.9750 - val_loss: 0.0357 - val_acc: 0.9876
Epoch 2/5
20000/20000 [==============================] - 204s - loss: 0.0608 - acc: 0.9785 - val_loss: 0.0360 - val_acc: 0.9878
Epoch 3/5
20000/20000 [==============================] - 203s - loss: 0.0634 - acc: 0.9775 - val_loss: 0.0356 - val_acc: 0.9872
Epoch 4/5
20000/20000 [==============================] - 204s - loss: 0.0598 - acc: 0.9789 - val_loss: 0.0349 - val_acc: 0.9878
Epoch 5/5
20000/20000 [==============================] - 204s - loss: 0.0618 - acc: 0.9784 - val_loss: 0.0349 - val_acc: 0.9876


In [8]:
model.save("models/xception_ft_model.h5")